In [15]:
import pandas as pd
import numpy as np
import re
from sympy import symbols

Function2

In [16]:
def salary_growing_by_year(state, occ_code, display=0):
    df = pd.read_csv('./filtered_state_M2022_dl.csv')
    result = df.loc[(df['AREA_TITLE'] == state) & (df['OCC_CODE'] == occ_code)]
    A_PCT10 = int(result['A_PCT10'].values[0])
    A_PCT25 = int(result['A_PCT25'].values[0])
    A_MEDIAN = int(result['A_MEDIAN'].values[0])
    if result['A_PCT75'].values[0] == '#':
#         print('##')
        A_PCT75 = 2*A_MEDIAN - A_PCT25
    else:
        A_PCT75 = int(result['A_PCT75'].values[0])

    if result['A_PCT90'].values[0] == '#':
#         print('##')
        A_PCT90 = 2*A_PCT75 - A_MEDIAN
    else:
        A_PCT90 = int(result['A_PCT90'].values[0])
    # A_PCT90
    x = np.array([0, 5, 10, 15, 20])
    y = np.array([A_PCT10, A_PCT25, A_MEDIAN, A_PCT75, A_PCT90])
    
    coeff = np.polyfit(x, y, 3)
    
    coeff = np.polyfit(x, y, 3)

    if display :
        # Generate values for the curve
        x_curve = np.linspace(0, 20, 100)
        poly_function = np.poly1d(coeff)
        y_curve = poly_function(x_curve)

        # Plotting the original data points and the fitted curve
        plt.figure(figsize=(8, 6))
        plt.scatter(x, y, label='Original Data Points')
        plt.plot(x_curve, y_curve, color='red', label='Fitted Curve')
        plt.xlabel('Years')
        plt.ylabel('Salaries')
        plt.title(f'Fitted Polynomial Curve for')
        plt.legend()
        plt.grid(True)
        plt.show()

    return coeff

In [17]:
# Function to find the tax rate
def find_tax_rate(state, income):
    
    new_file_path = 'filtered_income_tax_2022_manual.csv'  # Replace with your file path
    data = pd.read_csv(new_file_path)
    """
    Find the tax rate based on state and income.

    Args:
    state (str): The state name.
    income (float): The income amount.
    data (DataFrame): The DataFrame with tax information.

    Returns:
    float: The corresponding tax rate.
    """
    # Filter data for the specific state
    state_data = data[data['State'].str.contains(state, case=False, na=False)]

    # Sort by Bracket to ensure correct order
    state_data = state_data.sort_values(by='Bracket')

    # Initialize default rate
    rate = None

    # Find the correct bracket for the given income
    for index, row in state_data.iterrows():
        if income > row['Bracket']:
            rate = row['Rates']
        else:
            break

    return rate

### function3 estimate the year to pay off the house mortgage

In [34]:
def estimate_payoff_house_mortgage(occupation, state) :
    # mapping to the occ_code
    occupation_mapping = {
        'Management Occupations': '11-0000',
        'Business and Financial Operations Occupations': '13-0000',
        'Computer and Mathematical Occupations': '15-0000',
        'Architecture and Engineering Occupations': '17-0000',
        'Life, Physical, and Social Science Occupations': '19-0000',
        'Community and Social Service Occupations': '21-0000',
        'Legal Occupations': '23-0000',
        'Educational Instruction and Library Occupations': '25-0000',
        'Arts, Design, Entertainment, Sports, and Media Occupations': '27-0000',
        'Healthcare Practitioners and Technical Occupations': '29-0000',
        'Healthcare Support Occupations': '31-0000',
        'Protective Service Occupations': '33-0000',
        'Food Preparation and Serving Related Occupations': '35-0000',
        'Building and Grounds Cleaning and Maintenance Occupations': '37-0000',
        'Personal Care and Service Occupations': '39-0000',
        'Sales and Related Occupations': '41-0000',
        'Office and Administrative Support Occupations': '43-0000',
        'Farming, Fishing, and Forestry Occupations': '45-0000',
        'Construction and Extraction Occupations': '47-0000',
        'Installation, Maintenance, and Repair Occupations': '49-0000',
        'Production Occupations': '51-0000',
        'Transportation and Material Moving Occupations': '53-0000'
    }
    occ_code = occupation_mapping.get(occupation)
    
    
    # get the salary growing function
    coeff = salary_growing_by_year(state, occ_code, display=0)
    #build the function
    # Define the variables
    y = symbols('y')  # Year
    AI = coeff[0]*y**3 + coeff[1]*y**2 + coeff[2]*y + coeff[3]  # Annual income function (third-order polynomial)
    
    #some constants
    PIM = 0.25  # Percentage of Income for Mortgage (25%)
    MHP_percent = 0.95  # Mortgage Percentage of Housing Price (95%)
    FIR = 0.005  # Federal Interest Rate (0.5%)
    
    # find the real estate tax rate based on state
    df = pd.read_csv('real_estate_tax.csv')
    find_row = df.loc[(df['State'] == state)]
    tax = find_row['Real Estate Tax Rate'].values[0]
    RET = float(tax)/100  # Real estate tax rate (3%)
#     print(RET)
    #find state house price
    df = pd.read_csv('filtered_housing_prices.csv')
    find_row = df.loc[(df['State'] == state)]
    price = find_row['Mean of Median of housing price'].values[0]
    MHP = float(price)# Median Housing Price
    
    # Initial Mortgage Amount
    initial_mortgage = MHP * MHP_percent
    
    # Function to calculate annual mortgage payment for a given year
    def annual_mortgage_payment(year):
        income = AI.subs(y, year)
        # find the income tax
        state_income_tax = find_tax_rate(state, income)
#         print(type(state_income_tax))
#         print(state)
        federal_income_tax = find_tax_rate('Federal', income)
        
        IT = float(state_income_tax) +  float(federal_income_tax)
        
        after_tax_income = income * (1 - IT)  # Income after income tax
        after_all_taxes_income = after_tax_income - (MHP * RET)  # Income after all taxes
        return after_all_taxes_income * PIM
    
    # Calculating the number of years to pay off the mortgage
    remaining_mortgage = initial_mortgage
    year = 1
    while remaining_mortgage > 0:
        payment = annual_mortgage_payment(year)
        remaining_mortgage = remaining_mortgage + (remaining_mortgage * FIR) - payment
        year += 1

    return year - 1    
    
def estimate_payoff_house_mortgage_row(row):
    # Extract necessary information from the row
    state = row['AREA_TITLE']
    occ_title = row['OCC_TITLE']
    return estimate_payoff_house_mortgage(occ_title, state)

In [35]:
state = 'Texas'#'Washington'  #'California'
occupation = 'Computer and Mathematical Occupations'

print(estimate_payoff_house_mortgage(occupation, state))

17


In [36]:
import pandas as pd

def calculate_salary_impact(occupation_title, data):
    # Convert 'TOT_EMP' to numeric, setting errors to NaN
    data['TOT_EMP'] = pd.to_numeric(data['TOT_EMP'], errors='coerce')

    # Convert 'A_MEDIAN' to numeric, setting errors to NaN
    data['A_MEDIAN'] = pd.to_numeric(data['A_MEDIAN'], errors='coerce')

    # Filter the data for the specified occupation
    filtered_data = data[data['OCC_TITLE'] == occupation_title]

    # Exclude rows where 'TOT_EMP' or 'A_MEDIAN' is NaN
    filtered_data = filtered_data[filtered_data['TOT_EMP'].notna() & filtered_data['A_MEDIAN'].notna()]

    # Calculate total US employment for this occupation
    total_us_employment = filtered_data['TOT_EMP'].sum()

    # Calculate the percentage of employment for each state
    filtered_data['EMP_PERCENTAGE'] = (filtered_data['TOT_EMP'] / total_us_employment) * 100

    # Calculate the weighted median salary for each state based on employment percentage
    filtered_data['WEIGHTED_MEDIAN_SALARY'] = filtered_data['EMP_PERCENTAGE'] * filtered_data['A_MEDIAN'] / 100
    
#     return filtered_data.sort_values(by='WEIGHTED_MEDIAN_SALARY', ascending=False)

    return filtered_data[['AREA_TITLE', 'OCC_TITLE', 'TOT_EMP', 'EMP_PERCENTAGE', 'A_MEDIAN', 'WEIGHTED_MEDIAN_SALARY']].sort_values(by='WEIGHTED_MEDIAN_SALARY', ascending=False)

# Load your CSV file
file_path = 'filtered_state_M2022_dl.csv'  # Replace with the path to your CSV file
data = pd.read_csv(file_path)

# Specify the occupation title
occupation_title = "Computer and Mathematical Occupations"  # Replace with the desired occupation title

# Get the result
weighted_salary_result = calculate_salary_impact(occupation_title, data)
sorted_weighted_salary_result = calculate_salary_impact(occupation_title, data)
print(weighted_salary_result.head())  # This will display the first few rows of the result


      AREA_TITLE                              OCC_TITLE   TOT_EMP  \
95    California  Computer and Mathematical Occupations  695480.0   
992        Texas  Computer and Mathematical Occupations  444670.0   
739     New York  Computer and Mathematical Occupations  284680.0   
1061    Virginia  Computer and Mathematical Occupations  241590.0   
1084  Washington  Computer and Mathematical Occupations  204330.0   

      EMP_PERCENTAGE  A_MEDIAN  WEIGHTED_MEDIAN_SALARY  
95         13.898648    128690            17886.169938  
992         8.886398     94560             8402.977494  
739         5.689117    106360             6050.944815  
1061        4.827996    109910             5306.449897  
1084        4.083382    128220             5235.712778  


In [38]:
# Load your CSV file
file_path = 'filtered_state_M2022_dl.csv'  # Replace with the path to your CSV file
data = pd.read_csv(file_path)

def calculate_state_score(occupation_title):
    df = calculate_salary_impact(occupation_title, data)
#     print(df)
    df['estimated_years_to_pay_off_mortgages'] = df.apply(estimate_payoff_house_mortgage_row, axis=1)
#     df['estimated_years_to_pay_off_mortgages'] = estimate_payoff_house_mortgage(occupation_title, df['AREA_TITLE'])
    df['state_score'] = df['WEIGHTED_MEDIAN_SALARY']/df['estimated_years_to_pay_off_mortgages']
    return df[['AREA_TITLE', 'OCC_TITLE', 'WEIGHTED_MEDIAN_SALARY', 'estimated_years_to_pay_off_mortgages', 'state_score']].sort_values(by='state_score', ascending=False)

# Specify the occupation title
occupation_title = "Business and Financial Operations Occupations"

sorted_state_score = calculate_state_score(occupation_title)

print(sorted_state_score.head())

     AREA_TITLE                                      OCC_TITLE  \
738    New York  Business and Financial Operations Occupations   
94   California  Business and Financial Operations Occupations   
991       Texas  Business and Financial Operations Occupations   
209     Florida  Business and Financial Operations Occupations   
301    Illinois  Business and Financial Operations Occupations   

     WEIGHTED_MEDIAN_SALARY  estimated_years_to_pay_off_mortgages  state_score  
738             6838.404449                                    20   341.920222  
94              9946.809321                                    33   301.418464  
991             5857.367231                                    20   292.868362  
209             4472.369628                                    23   194.450853  
301             3133.094128                                    17   184.299655  
